In [1]:
import os
import cv2
import numpy as np
import rootutils
from pathlib import Path

rootutils.setup_root(
    os.path.abspath(''), indicator=['.git', 'pyproject.toml'], pythonpath=True
)

from src.data.components.utils import find_file_by_name

In [2]:
def blur_background(image: np.array, background_mask: np.array) -> np.array:
    if background_mask is None or background_mask.size == 0:
        raise ValueError("The provided mask is empty or invalid.")
    blurred = cv2.GaussianBlur(image, (61, 61), 100)
    _, binary_mask = cv2.threshold(background_mask, 128, 255, cv2.THRESH_BINARY)
    binary_mask = cv2.cvtColor(binary_mask, cv2.COLOR_GRAY2BGR)
    inverted_mask = cv2.bitwise_not(binary_mask)
    blurred_outside = cv2.bitwise_and(blurred, inverted_mask)
    original_inside = cv2.bitwise_and(image, binary_mask)
    result = cv2.add(original_inside, blurred_outside)

    return result

In [7]:
file_names = [
    '20231004225948-024130155-AC FS Camera Left 1',
    '20231004225947-024130155-AC FS Camera Left 2',
    '20231004225947-024130155-AC FS Camera Right 1',
    '20231004225947-024130155-AC FS Camera Right 2'
]

data_path = Path(os.environ.get('lear_wrinkles_data_path'))
image_path = data_path / 'Photos'
mask_path = data_path / 'Outline'

output_path = Path('output2/')
output_path.mkdir(parents=True, exist_ok=True)

for file_name in file_names:
    image = cv2.imread(find_file_by_name(image_path, file_name), cv2.IMREAD_COLOR)
    mask = cv2.imread(find_file_by_name(mask_path, file_name + '_mask'), cv2.IMREAD_GRAYSCALE)
    blurred_image = blur_background(image, mask)
    cv2.imwrite(output_path / (file_name + '.png'), blurred_image)